Importando bibliotecas

In [1]:
import pandas as pd
import json
import gensim
import nltk
from  nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts, get_tmpfile
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn import tree
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.manifold import TSNE
from bokeh.models import ColumnDataSource, LabelSet, LayoutDOM, Plot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.core.properties import Instance, String
from bokeh.resources import INLINE
import bokeh.io

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Importando base concatenada e misturando os artigos para evitar que haja um viés devido a ordem de leitura dos documentos.

In [35]:
abstract = pd.read_json("Elsevier_abstract - Consolidado.json")
#Misturando as linhas e resetando o index
abstract = abstract.sample(frac=1).reset_index(drop=True)

## Preprocessamento

Retirando palavras e espaços

In [36]:
# Retirando as palavras "Abstract", "Unknown", "Publisher Summary", "Summary", "Fundamento"
# que aparece no início de vários textos
abstract["Abstract"] = abstract["Abstract"].str.replace("Abstract", "")
abstract["Abstract"] = abstract["Abstract"].str.replace("Unknown", "")
abstract["Abstract"] = abstract["Abstract"].str.replace("Publisher Summary", "")
abstract["Abstract"] = abstract["Abstract"].str.replace("Summary", "")
abstract["Abstract"] = abstract["Abstract"].str.replace("Fundamento", "")

In [37]:
# Separando o texto em uma lista de palavras desconsiderando espaços e \n
abstract["Abstract"] = abstract["Abstract"].str.split()      
# Unificando a lista de palavras usando espaços simples
abstract["Abstract"] = abstract["Abstract"].str.join(" ")

Eliminando as linhas sem Abstract

In [38]:
abstract = abstract[abstract["Abstract"] != ""]
abstract = abstract.reset_index()
abstract = abstract.drop(columns=['index'])

Realizando o pré-processamento simple do Gensim basicamente para gerar os tokens dos textos

In [39]:
abstract["Abstract"] = abstract["Abstract"].apply(gensim.utils.simple_preprocess)

Gravando arquivo com os documentos pré-processados

In [70]:
abstract.to_json("Elsevier_abstract - Preprocessado.json")
abstract = pd.read_json("Elsevier_abstract - Preprocessado.json")

In [40]:
print ("Artigos no qual um dos autores é afiliado a empresas de O&G: ",len(abstract[abstract["O&G"] == 1]))
print ("Artigos em que nenhum dos autores é afiliado a empresas de O&G: ",len(abstract[abstract["O&G"] == 0]))

Artigos no qual um dos autores é afiliado a empresas de O&G:  14418
Artigos em que nenhum dos autores é afiliado a empresas de O&G:  11227


Dividindo os dados usando 90% para treino e 10% para teste.

In [41]:
# Dividindo os dados em treino e teste
abstract_train, abstract_test = train_test_split(abstract, test_size=0.1)
print ("Conjunto de Treino")
print ("Artigos no qual um dos autores é afiliado a empresas de O&G: ",
       len(abstract_train[abstract_train["O&G"] == 1]))
print ("Artigos em que nenhum dos autores é afiliado a empresas de O&G: ",
       len(abstract_train[abstract_train["O&G"] == 0]))
print ("")
print ("Conjunto de Teste")
print ("Artigos no qual um dos autores é afiliado a empresas de O&G: ",
       len(abstract_test[abstract_test["O&G"] == 1]))
print ("Artigos em que nenhum dos autores é afiliado a empresas de O&G: ",
       len(abstract_test[abstract_test["O&G"] == 0]))

Conjunto de Treino
Artigos no qual um dos autores é afiliado a empresas de O&G:  12962
Artigos em que nenhum dos autores é afiliado a empresas de O&G:  10118

Conjunto de Teste
Artigos no qual um dos autores é afiliado a empresas de O&G:  1456
Artigos em que nenhum dos autores é afiliado a empresas de O&G:  1109


Aumentando os dados de treinamento deixando uma proporção de 2/3 dos artigos de O&G e 1/3 para não O&G.

In [195]:
#augmented_abstract_train = abstract_train
# Triplicando os artigos de O&G
#augmented_abstract_train = augmented_abstract_train.append(abstract_train[abstract_train["O&G"]==1],
#                                                           ignore_index=True)
#augmented_abstract_train = augmented_abstract_train.append(abstract_train[abstract_train["O&G"]==1],
#                                                           ignore_index=True)
# Duplicando os artigos de que não são O&G
#augmented_abstract_train = augmented_abstract_train.append(abstract_train[abstract_train["O&G"]==0],
#                                                           ignore_index=True)
# Embaralhando os documentos
#augmented_abstract_train = augmented_abstract_train.sample(frac=1).reset_index(drop=True)

#print ("Conjunto de Treino Aumentado")
#print ("Artigos no qual um dos autores é afiliado a empresas de O&G: ", 
#       len(augmented_abstract_train[augmented_abstract_train["O&G"] == 1]))
#print ("Artigos em que nenhum dos autores é afiliado a empresas de O&G: ",
#       len(augmented_abstract_train[augmented_abstract_train["O&G"] == 0]))
#print ("")

Conjunto de Treino Aumentado
Artigos no qual um dos autores é afiliado a empresas de O&G:  25876
Artigos em que nenhum dos autores é afiliado a empresas de O&G:  30426



## Vetorizando e classificando os documentos

Gerando os modelos Doc2Vec utilizando os artigos provenientes do conjunto de treino

In [42]:
momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

# Todos os documentos devem passar pela função TaggedDocument antes de ser vetorizado
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(abstract_train["Abstract"])]
#documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(augmented_abstract_train["Abstract"])]
#Hiperparâmetros do modelo Doc2Vec
Abstract_doc2vec = Doc2Vec(documents,
                           vector_size=50,
                           dm=0,
                           window=5,
                           min_count=1,
                           workers=7,
                           epochs=100,
                           alpha=0.025,
                           min_alpha=0.00025)

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

Tempo total decorrido:  0:03:45.266000


In [43]:
# Gravando modelo Doc2Vec
fname = "Abstract_doc2vec_model"
Abstract_doc2vec.save(fname)

In [44]:
# Lendo modelo Doc2Vec
fname = "Abstract_doc2vec_model"
Abstract_doc2vec = Doc2Vec.load(fname)

In [45]:
# Utilizar o modelo Doc2Vec gerado para inferir os vetores para todos os artigos.

momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

abstract["Doc2Vec"] = abstract["Abstract"].apply(Abstract_doc2vec.infer_vector)

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

Tempo total decorrido:  0:06:33.761800


In [46]:
# Utilizar o modelo Doc2Vec gerado para inferir 
# os vetores para todos os abstract_train e abstract_test

momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

abstract_train["Doc2Vec"] = abstract_train["Abstract"].apply(Abstract_doc2vec.infer_vector)
#augmented_abstract_train["Doc2Vec"] = augmented_abstract_train["Abstract"].apply(Abstract_doc2vec.infer_vector)
abstract_test["Doc2Vec"] = abstract_test["Abstract"].apply(Abstract_doc2vec.infer_vector)

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Tempo total decorrido:  0:06:41.745800


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Modelo de classificação

In [47]:
# Separando os vetores com atributos e as classes 
features_train = np.array(abstract_train["Doc2Vec"].tolist())
#features_train = np.array(augmented_abstract_train["Doc2Vec"].tolist())
features_test = np.array(abstract_test["Doc2Vec"].tolist())
label_train = np.array(abstract_train["O&G"])
#label_train = np.array(augmented_abstract_train["O&G"])
label_test = np.array(abstract_test["O&G"])

Treinando uma Árvore de Decisão para identificar as features mais importantes

In [48]:
# Treinando uma Decision Tree com todas as 50 dimensões 
# como atributos para a classificação

momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

clf = tree.DecisionTreeClassifier()
clf.fit(features_train, label_train)        # Treinando o classificador

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

Tempo total decorrido:  0:00:01.995000


Usando para o treinamento apenas os "N" atributos mais importantes

In [49]:
# Número de atributos que será usado para treinamento
N = 50    
# Usando classificador decision tree para achar os atributos mais importantes
important = np.argsort(clf.feature_importances_).tolist()
important = important[-N:]

#Usando apenas os astributos mais importantes
features_train = pd.DataFrame(features_train)[important]
features_test = pd.DataFrame(features_test)[important]

In [50]:
# Treinando uma Suport Vector Machine (SVM) com todas as 100 dimensões 
# como atributos para a classificação

momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

# Métricas de avaliação
f1 = []
acc = []
precision = []
recall = []

# Criando os classificadores
#clf = svm.SVC()                                  
#clf = RandomForestClassifier(n_estimators=100)   
#clf = tree.DecisionTreeClassifier()
#clf = NearestCentroid()
#clf = KNeighborsClassifier()
#clf = GaussianNB()
#clf = MLPClassifier(solver='sgd')
clf = MLPClassifier(solver='sgd', hidden_layer_sizes=5000 )

clf.fit(features_train, label_train)        # Treinando o classificador
# Predizendo o conjunto de teste com base do classificador treinado
label_pred = clf.predict(features_test)     
    
# Definindo as métricas F1, Acurácia, Precisão e Revocação
precision, recall, f1, support = precision_recall_fscore_support(label_test,
                                                                 label_pred,
                                                                 average='weighted')
acc = accuracy_score(label_test,
                     label_pred)

# Criando um DataFrame com os resultados
scores = {'F1-Score':f1,
          'Acurácia':acc,
          'Precisão':precision,
          'Revocação':recall}
scores = pd.DataFrame.from_dict(scores, orient='index')

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

scores

Tempo total decorrido:  0:14:47.819000


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,0
F1-Score,0.864836
Acurácia,0.865107
Precisão,0.864874
Revocação,0.865107
